In [1]:
using Pkg
Pkg.add("Plots")
Pkg.add("DifferentialEquations")
Pkg.add("SpecialFunctions")

    Updating registry at `C:\Users\Seiven\.julia\registries\General.toml`


   Resolving package versions...


  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Seiven\.julia\environments\v1.8\Manifest.toml`

In [6]:
import Pkg;
Pkg.add("FastSphericalHarmonics");

   Resolving package versions...

   Installed ComputedFieldTypes ───── v1.0.2


   Installed FastTransforms_jll ───── v0.6.2+0
   Installed GenericFFT ───────────── v0.1.6
   Installed ToeplitzMatrices ─────── v0.8.3


   Installed FastSphericalHarmonics ─ v1.3.1


   Installed FastTransforms ───────── v0.15.16
   Installed DSP ──────────────────── v0.7.9
   Installed FastGaussQuadrature ──── v1.0.2


...

  ✓ FastGaussQuadrature


  ✓ DSP


  ✓ ToeplitzMatrices


  ✓ FastTransforms


  ✓ FastSphericalHarmonics
  10 dependencies successfully precompiled in 19 seconds. 396 already precompiled.


In [7]:
using LinearAlgebra
using Plots
using DifferentialEquations
using FastSphericalHarmonics

# Parameters
l_max = 4  # Maximum degree of spherical harmonics
sigma = 0.2  # Width of the initial condition
t_max = 10  # Maximum time
n_points = 100  # Number of points for theta and phi

# Define theta and phi arrays
theta = range(0, stop=π, length=n_points)
phi = range(0, stop=2π, length=n_points)

# Custom meshgrid function for Julia
function meshgrid(x, y)
    X = repeat(reshape(x, 1, :), length(y), 1)
    Y = repeat(reshape(y, :, 1), 1, length(x))
    return X, Y
end

Theta, Phi = meshgrid(theta, phi)

# Initial condition function
function initial_condition(theta, sigma)
    return exp.(-theta .^ 2 / (2 * sigma^2))
end

# Project initial condition onto spherical harmonics basis
function project_initial_condition(l, m, sigma, theta, phi)
    integral = 0.0
    for theta_val in theta
        for phi_val in phi
            # Adjust this call to match the actual spherical harmonics function
            Y_lm = conj(FastSphericalHarmonics.sph_harm(m, l, phi_val, theta_val))
            integral += initial_condition(theta_val, sigma) * Y_lm * sin(theta_val)
        end
    end
    return integral * (theta[2] - theta[1]) * (phi[2] - phi[1])
end


# ODE system for c_lm(t) and psi_lm(t)
function ode_system(du, u, p, t)
    l_max = p
    for l in 0:l_max
        for m in -l:l
            idx = l^2 + l + m + 1  # Adjust index for 1-based indexing
            du[idx] = u[idx+l_max^2+l_max]  # d/dt c_lm(t) = psi_lm(t)
            du[idx+l_max^2+l_max] = -l * (l + 1) * u[idx]  # d/dt psi_lm(t) = -l(l+1)c_lm(t)
        end
    end
end

# Solve the ODE system
function solve_ode(l_max, sigma, t_max, theta, phi)
    # Number of coefficients
    n_coeffs = (l_max + 1)^2
    # Initial conditions for c_lm(0) and psi_lm(0)
    y0 = zeros(Float64, 2 * n_coeffs)
    for l in 0:l_max
        for m in -l:l
            idx = l^2 + l + m + 1  # Adjust index for 1-based indexing
            y0[idx] = project_initial_condition(l, m, sigma, theta, phi)
        end
    end
    # Time span
    tspan = (0.0, t_max)
    # Solve ODE
    prob = ODEProblem(ode_system, y0, tspan, l_max)
    sol = solve(prob, Tsit5(), reltol=1e-8)
    return sol
end

# Visualization of the solution at a given time t
function visualize_solution(sol, t, l_max, theta, phi)
    Theta_plot, Phi_plot = meshgrid(theta, phi)
    Z = zeros(Float64, length(theta), length(phi))
    for l in 0:l_max
        for m in -l:l
            idx = l^2 + l + m + 1  # Adjust index for 1-based indexing
            c_lm = sol(t)[idx]
            for (i, theta_val) in enumerate(theta)
                for (j, phi_val) in enumerate(phi)
                    Z[i, j] += c_lm * real(sph_harm(m, l, phi_val, theta_val))
                end
            end
        end
    end
    p = contourf(Phi_plot', Theta_plot', Z, title="Solution at t=$t", xlabel="Phi", ylabel="Theta", color=:viridis)
    display(p)
end

# Main execution
l_max_values = [4, 8, 12]  # Different l_max values for comparison
for l_max_val in l_max_values
    sol = solve_ode(l_max_val, sigma, t_max, theta, phi)
    # Visualize at different time steps
    for t in range(0, stop=t_max, length=5)
        visualize_solution(sol, t, l_max_val, theta, phi)
    end
end


UndefVarError: UndefVarError: sph_harm not defined